In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import torch.nn.functional as F

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

from synth import CrossEntropyDifferential

/home/tuccio/synth/menv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
device = torch.device("cuda")

model_name = "microsoft/Phi-3.5-mini-instruct"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
    attn_implementation="eager"
    )

tokenizer = AutoTokenizer.from_pretrained(model_name)

cross_entropy_differential = CrossEntropyDifferential(model, tokenizer, device)

`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.
Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.07it/s]


In [4]:
Y = """ thing in the world, all the mozzarella and the tomato sauce on my margherita are amazing. 
My Italian friend Angelo told me that his favourite pizza is the quattroformaggi rossa with salsiccia. 
When I was a kid, I used to watch the pizza maker create his pizzas, he was from Romania but a very nice gentleman I have to say."""

message = [
    {"role": "system", "content":"Write a small phrase that would fit well at the ::START:: the phrase"},
    {"role": "user", "content":Y}
]

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

generation_args = {
    "max_new_tokens": 20,
    "return_full_text": False,
    "do_sample": True,
    "temperature": 1
}

output = pipe(message, **generation_args)

The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
You are not running the flash-attention implementation, expect numerical differences.


In [10]:
print(len(output[0]["generated_text"]))

result = (
cross_entropy_differential(output[0]["generated_text"], Y, diff=True).item(),
output[0]["generated_text"]
    )
result

61
21


ValueError: Expected input batch_size (93) to match target batch_size (94).

In [7]:
results = []
for i in output:
    answ = tokenizer.decode(i[len(input.input_ids[0]):], skip_special_tokens=True)
    results.append((
        cross_entropy_differential(answ, Y, diff=True).item(),
        answ
        ))
best_result = min(results, key=lambda x: x[0])
best_result

AttributeError: 'function' object has no attribute 'input_ids'